In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pyabf
import glob
import seaborn as sns
import pandas as pd
import os
import scipy.stats as stats

In [16]:
def baseline_sweep(abf):
    #baseline based on the slope
    params = stats.linregress(abf.sweepX, abf.sweepY)
    baseline = params.intercept + params.slope * abf.sweepX
    data_out = abf.sweepY - baseline
    return data_out


In [17]:
MAIN_FOLDER = "Z:\\Molsrv\\Julia\\Data\\Opto\\Opto Perifornical for Grant_2025\\ALL FILES"
MAIN_FILE = "Z:\\Molsrv\\Julia\\Data\\Opto\\Opto Perifornical for Grant_2025\\FILENAMES_Opto Perifornical.xlsx"

main_df = pd.read_excel(MAIN_FILE)
print(main_df.tail())

     Cell ID    Date Mouse ID Sex Condition Slice ID Well#     Filled?  \
232  6_SM_12  250815   867B-3   M        KD      4-R    D8  Not filled   
233  6_SM_13  250815   867B-3   M        KD      4-R    D8  Not filled   
234  6_SM_14  250815   867B-3   M        KD      4-R    D8  Not filled   
235  6_SM_15  250815   867B-3   M        KD      4-R    D8  Not filled   
236  6_SM_16  250815   867B-3   M        KD      4-R    D8  Not filled   

                Area Exerimenter  ...  PP (20 ms)   PP Link PS Check VC  \
232       PF-lateral          SM  ...    25815047  25815047    25815048   
233  PF-ventromedial          SM  ...    25815051  25815051    25815052   
234              PVN          SM  ...    25815055  25815055    25815056   
235              PVN          SM  ...    25815059  25815059    25815060   
236   PF-dorsomedial          SM  ...    25815063  25815063    25815064   

     PS CheckVC  Link PS Check IC PS Check IC Link 10 Hz Train  \
232          25815048         NaN     

In [18]:

abf_files = glob.glob(MAIN_FOLDER + "\\*.abf")
all_results = {}
PRE_TIME = (1.027*1000, 1.047*1000)
POST_TIME = (1.047*1000, 1.062*1000)
col_map = {0: "sweep", 4: "event_time", 7:"peak_amp"}
col_map_inv = {v: k for k, v in col_map.items()}
RAW_DATA = {}
for index, row in main_df.iterrows():
    fe = MAIN_FOLDER + "\\" + str(row["PP (20 ms)"]) + ".abf"
    if os.path.exists(fe):
        abf = pyabf.ABF(fe, loadData=False)
        print(f"Loaded {abf.abfID} with {abf.sweepCount} sweeps")
        #actually load it
        abf = pyabf.ABF(fe)
    else:
        print(f"File {fe} not found.")
        continue

    results = {"Pre": [], "Post": [], 'max': []}
    for sweep in np.arange(abf.sweepCount):
        
        abf.setSweep(sweep)
        idx_start = np.argmin(np.abs(abf.sweepX - POST_TIME[0]/1000))
        idx_end = np.argmin(np.abs(abf.sweepX - POST_TIME[1]/1000))
        sweep_baselined = baseline_sweep(abf)
      
        max_peak = np.min(sweep_baselined[idx_start:idx_end])
        
        results["max"].append(max_peak)

    
    results["max_mean"] = np.mean(results['max'])


    print(f"Updating row {index} with results: {results}")
    main_df.loc[index, "max_mean"] = results["max_mean"]
    for i in range(len(results["max"])):
        main_df.loc[index, f"max_sweep_{i}"] = results["max"][i]


Loaded 2025_08_06_0003 with 18 sweeps
Updating row 0 with results: {'Pre': [], 'Post': [], 'max': [-5.597978124145339, -4.823073429816919, -3.943025647278322, -3.609112395047081, -3.20826354685488, -3.8239734948842106, -4.240968971075535, -3.3006162205469423, -34.56995344191779, -4.6918560509464555, -5.249454959960929, -5.436805490671098, -3.4256474440991695, -6.851976210240057, -41.86825480811798, -3.598075400476919, -3.335760398768425, -4.676437516258908], 'max_mean': -8.125068530617053}
Loaded 2025_08_06_0007 with 19 sweeps
Updating row 1 with results: {'Pre': [], 'Post': [], 'max': [-9.083935778817192, -10.622630157107274, -3.0270641674206047, -2.858322510293501, -3.0955783013763494, -6.339967285327749, -8.720763291223104, -12.682977418957961, -13.299028016021417, -5.261400327324054, -3.926412319723461, -2.976840905231178, -8.247748650335566, -8.79242394129156, -8.147115675414032, -2.9954091448471125, -12.27230020133105, -9.750240631843397, -5.3187708190493055], 'max_mean': -7.2325

In [19]:
main_df.to_csv("file_names_with_peaks.csv", index=False)